<h2 align="center">Codebasics DL Course: Vehicle Damage Detection Project: Hyperparameter Tunning</h2>

In [1]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import time
import torchvision.models as models
from matplotlib import pyplot as plt
import optuna

In [2]:
device = torch.device("mps" if torch.mps.is_available() else "cpu")
device

device(type='mps')

### Load Data

In [3]:
image_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
dataset_path = "./dataset"

dataset = datasets.ImageFolder(root=dataset_path, transform=image_transforms)
len(dataset)

2664

In [5]:
2664*0.8

2131.2000000000003

In [6]:
class_names = dataset.classes
class_names 

['F_Breakage',
 'F_Crushed',
 'F_Normal',
 'R_Breakage',
 'R_Crushed',
 'R_Normal',
 'S_Damaged',
 'S_Normal']

In [7]:
num_classes = len(dataset.classes)
num_classes

8

In [8]:
train_size = int(0.8*len(dataset))
val_size = len(dataset) - train_size

train_size, val_size

(2131, 533)

In [9]:
from torch.utils.data import random_split

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [10]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)

### Model Training & Hyperparameter Tunning

In [11]:
# Load the pre-trained ResNet model
class CarClassifierResNet(nn.Module):
    def __init__(self, num_classes, dropout_rate=0.5):
        super().__init__()
        self.model = models.resnet50(weights='DEFAULT')
        # Freeze all layers except the final fully connected layer
        for param in self.model.parameters():
            param.requires_grad = False
            
        # Unfreeze layer4 and fc layers
        for param in self.model.layer4.parameters():
            param.requires_grad = True            
            
        # Replace the final fully connected layer
        self.model.fc = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(self.model.fc.in_features, num_classes)
        )

    def forward(self, x):
        x = self.model(x)
        return x

In [12]:
def objective(trial):
    # Suggest values for the hyperparameters
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log=True)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.6)

    # Load the model
    model = CarClassifierResNet(num_classes=num_classes, dropout_rate=dropout_rate).to(device)

    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)

    # Training loop
    max_epochs = 10
    patience = 3  # Early stopping patience
    best_accuracy = 0.0
    epochs_no_improve = 0

    start = time.time()
    for epoch in range(max_epochs):
        model.train()
        running_loss = 0.0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)

        # Validation
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        trial.report(accuracy, epoch)

        # Optuna pruning
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        # Early stopping
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

    end = time.time()
    print(f"Execution time: {end - start:.2f} seconds")

    return best_accuracy

In [13]:
# Create the study and optimize
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=8)

[I 2025-07-13 18:12:29,513] A new study created in memory with name: no-name-87e755d4-d6ee-4bb5-8a53-31fadc6a8278
[I 2025-07-13 18:29:25,800] Trial 0 finished with value: 85.92870544090056 and parameters: {'lr': 0.0024012311531690475, 'dropout_rate': 0.29231861632437395}. Best is trial 0 with value: 85.92870544090056.


Execution time: 1015.80 seconds


[I 2025-07-13 18:45:02,271] Trial 1 finished with value: 84.24015009380864 and parameters: {'lr': 0.0007755208380911534, 'dropout_rate': 0.2228590258892009}. Best is trial 0 with value: 85.92870544090056.


Execution time: 935.81 seconds


[I 2025-07-13 18:55:58,664] Trial 2 finished with value: 83.67729831144466 and parameters: {'lr': 0.008903823826109348, 'dropout_rate': 0.468035644223998}. Best is trial 0 with value: 85.92870544090056.


...

[I 2025-07-13 19:08:07,361] Trial 3 finished with value: 83.86491557223265 and parameters: {'lr': 0.00017900540937140674, 'dropout_rate': 0.2604398956009095}. Best is trial 0 with value: 85.92870544090056.


Early stopping at epoch 8
Execution time: 728.18 seconds


[I 2025-07-13 19:24:14,032] Trial 4 finished with value: 76.92307692307692 and parameters: {'lr': 1.1488852721047999e-05, 'dropout_rate': 0.27591744866792556}. Best is trial 0 with value: 85.92870544090056.


Execution time: 966.14 seconds


[I 2025-07-13 19:26:08,131] Trial 5 pruned. 
[I 2025-07-13 19:27:48,975] Trial 6 pruned. 
[I 2025-07-13 19:29:29,189] Trial 7 pruned. 


In [14]:
study.best_params

{'lr': 0.0024012311531690475, 'dropout_rate': 0.29231861632437395}